In [10]:
import pandas as pd
import os
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship

In [3]:
proc_df = pd.read_csv('./Data/CSVs/proc.csv', header=None)
proc_df.head()

,1,C1$@DOM1,C1,P16,Start
0,1,C1001$@DOM1,C1001,P4,Start
1,1,C1002$@DOM1,C1002,P4,Start
2,1,C1004$@DOM1,C1004,P4,Start
3,1,C1017$@DOM1,C1017,P4,Start
4,1,C1018$@DOM1,C1018,P4,Start


In [4]:
redteam_df = pd.read_csv('./Data/CSVs/redteam.csv', header=None)
redteam_df.head()

,0,1,2,3
0,150885,U620@DOM1,C17693,C1003
1,151036,U748@DOM1,C17693,C305
2,151648,U748@DOM1,C17693,C728
3,151993,U6115@DOM1,C17693,C1173
4,153792,U636@DOM1,C17693,C294


In [5]:
dns_df = pd.read_csv('./Data/CSVs/dns.csv', header=None)
dns_df.head()

,0,1,2
0,2,C4653,C5030
1,2,C5782,C16712
2,6,C1191,C419
3,15,C3380,C22841
4,18,C2436,C5030


In [7]:
flows_df = pd.read_csv('./Data/Csvs/flows.csv', header=None)
flows_df.head()

MemoryError: 

In [11]:
# create engine to cybersec sqlite database
engine = create_engine("sqlite:///sec_data.sqlite")

In [12]:
# declare the base
Base = declarative_base()

In [15]:
# create a class for redteam data
class redTeam(Base):
    __tablename__ = "redteam_data"
    id = Column(Integer, primary_key=True)
    time = Column(Float)
    userDomain = Column(Text)
    sourceComputer = Column(Text)
    destinationComputer = Column(Text)
    
    def __repr__(self):
        return f"id={self.id}, name={self.userDomain}"

In [16]:
# create a class for DNS data
class DNS(Base):
    __tablename__ = "dns_data"
    id = Column(Integer, primary_key=True)
    time = Column(Float)
    sourceComputer = Column(Text)
    computerResolved = Column(Text)
    
    def __repr__(self):
        return f"id={self.id}, name={self.sourceComputer}"

In [17]:
# create class for network flow events data
class networkFlow(Base):
    __tablename__ = "flow_data"
    id = Column(Integer, primary_key=True)
    time = Column(Float)
    duration = Column(Float)
    sourceComputer = Column(Text)
    sourcePort = Column(Text)
    destinationComputer = Column(Text)
    destinationPort = Column(Text)
    protocol = Column(Float)
    packetCount = Column(Float)
    byteCount = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.sourceComputer}"

In [18]:
# create class for proc data
class proc(Base):
    __tablename__ = "proc_data"
    id = Column(Integer, primary_key=True)
    time = Column(Float)
    userDomain = Column(Text)
    computer = Column(Text)
    processName = Column(Text)
    startEnd = Column(Text)
    
    def __repr__(self):
        return f"id={self.id}, name={self.userDomain}"

In [19]:
# create all tables
Base.metadata.create_all(engine)

In [20]:
# verify that tables are created
engine.table_names()

['dns_data', 'flow_data', 'proc_data', 'redteam_data']

In [23]:
# function to insert the csv file data into the sqlite tables
def populate_table(engine, table, csvfile):
    # connect to engine
    conn = engine.connect()
    
    #pandas reads csv file
    df_data = pd.read_csv(csvfile)
    
    # create dict of records to write
    data = df_data.to_dict(orient="records")
    
    #deletes all if records already exist
    conn.execute(table.delete())
    
    # pushes the entire dataset
    conn.execute(table.insert(),data)

In [24]:
populate_table(engine, redTeam.__table__, './Data/CSVs/redteam.csv')
populate_table(engine, DNS.__table__, './Data/CSVs/dns.csv')

StatementError: (builtins.MemoryError)  [SQL: 'INSERT INTO dns_data DEFAULT VALUES'] [parameters: [{'2': 2, 'C4653': 'C5782', 'C5030': 'C16712'}, {'2': 6, 'C4653': 'C1191', 'C5030': 'C419'}, {'2': 15, 'C4653': 'C3380', 'C5030': 'C22841'}, {'2': 18, 'C4653': 'C2436', 'C5030': 'C5030'}, {'2': 31, 'C4653': 'C161', 'C5030': 'C2109'}, {'2': 35, 'C4653': 'C5642', 'C5030': 'C528'}, {'2': 38, 'C4653': 'C3380', 'C5030': 'C22841'}, {'2': 42, 'C4653': 'C2428', 'C5030': 'C1065'}  ... displaying 10 of 40821590 total bound parameter sets ...  {'2': 5011199, 'C4653': 'C743', 'C5030': 'C586'}, {'2': 5011199, 'C4653': 'C893', 'C5030': 'C2174'}]]